In [ ]:
import random
from random import randint
import operator
from typing import List, Callable, Tuple
from collections import Counter 
from prettytable import PrettyTable

In [ ]:
def make_one_schedules(length,Section,Teacher,Room,Course,Day,Duration):
    arr=[]
    for i in range(0,length):
        arr.append([random.choice(Teacher),random.choice(Course),random.choice(Section),random.choice(Day),random.choice(Duration),random.choice(Room)])
    return arr
def initialize_population(size,length,Section,Teacher,Room,Course,Day,Duration):
    for i in range(0,size):
        schedules.append(make_one_schedules(length,Section,Teacher,Room,Course,Day,Duration))
    return schedules

In [ ]:
def evaluate_fitness(schedule):
    fitness=0
    conflict=0
    h1=0
    h2=0
    h3=0
    h4=0
    h5=0
    s1=0
    for i in range(0,len(schedule)-1):
        if(schedule[i][4]=='07:00 AM' or schedule[i][4]=='05:00 PM' or schedule[i][4]=='05:30 PM'): #Hard--There will be no class before 8:30 am and after 5:00 pm.
            h1=h1+1
        
        if(schedule[i][4]=='01:00 PM' and schedule[i][3]=='Friday'):  #Soft--There will be no class from 1-2 on Friday.
            s1=s1+1

        if(schedule[i][3]=='Sunday' or schedule[i][3]=='Saturday'): #Hard--University will remain close as there will be no class on weekends (Sat, Sun)
            h2=h2+1

    j=1
                                      #Hard--No teacher can hold two classes at the same time
    for k in range(0,7):
        if(j==7):
            break
        else:
            for j in range(k+1,7):
                if(schedule[k][0] ==schedule[j][0] and schedule[k][4]== schedule[j][4] and schedule[k][3]== schedule[j][3]):
                    h3=h3+1


    j=1
    clashes=dict ()                             #Hard--No classroom can receive two classes at the same time
    for k in range(0,7):
        if(j==7):
            break
        else:
            for j in range(k+1,7):
                if(schedule[k][4] ==schedule[j][4] and schedule[k][5]== schedule[j][5] and schedule[k][3]== schedule[j][3]):
                    h4=h4+1

    j=1
    Section_listen=dict ()                             #Hard--No section can listen for two classes at the same time
    for k in range(0,7):
        if(j==7):
            break
        else:
            for j in range(k+1,7):
                if(schedule[k][4] ==schedule[j][4] and schedule[k][3]== schedule[j][3] and schedule[k][2]== schedule[j][2]):
                    h5=h5+1

    conflict=h1+h2+h3+h4+h5+s1
    if (conflict==0):
        fitness=3
    else:
        fitness=(1/conflict)
    return fitness



In [ ]:
def roulette_wheel(schedules):
    cal_fit=dict ()
    p=[]
    fitness_sum=0
    for i in range(0,len(schedules)):
        f=evaluate_fitness(schedules[i])
        p.append(f)
        cal_fit[i]=f
        fitness_sum+=evaluate_fitness(schedules[i])
    
    sorted_d = dict (sorted(cal_fit.items(), key=operator.itemgetter(1))) #value is fitness and key is schedule
    
    r = random.uniform(0,fitness_sum)
    partial_sum = 0
    for i in range(len(schedules)): #Roulette Selection Complete
        partial_sum += p[i]
        if partial_sum > r:
            return schedules[i]


In [ ]:
def single_point_crossover(a, b):
    if len(a) != len(b):
        raise ValueError("Both Parents a and b must be of equal length")
    length = len(a[0])
    if len(a)<2:
        split = randint(1, length-1)
        return a[0:split] + b[split:], b[0:split] + a[split:]
    
    split = randint(1, length-1)
    return a[0:split] + b[split:], b[0:split] + a[split:]

In [ ]:
def mutation(child,section,teacher,room,course,day,duration):
    Mutation_rate=0.5
    m=random.random()
    if(m<Mutation_rate): #no mutation
        return child
    r1=randint(0,5)
    r2=randint(0,len(child)-1)
    if (r1==0):
        child[r2][0]=random.choice(teacher)
    elif (r1==1):
        child[r2][1]=random.choice(course)
    elif (r1==2):
        child[r2][2]=random.choice(section)
    elif (r1==3):
        child[r2][3]=random.choice(day)
    elif (r1==4):
        child[r2][4]=random.choice(duration)
    elif (r1==5):
        child[r2][5]=random.choice(room)
    
    return child

In [ ]:
def genetic_algorithm(size,length,generation_limit):
    # Data 
    Section=['A','B','C','D','E']
    Teacher=['T1- Sir Bilal Khalid','T2- Sir Kashif Munir','T3- Maam Amna Basharat','T4- Dr Omer Beg','T5- Dr Irum Inayat']
    Course=['330 Maths','440 English','331 Intro to Computing','335 Applied Physics','160 Bioinformatics']
    Day=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    Duration=['07:00 AM','08:30 AM','09:30 AM','11:00 AM','12:00 PM','01:00 PM','02:00 PM','03:00 PM','04:00 PM','05:00 PM']
    Room=['R1','R2','R3','R4','R5']
    schedule=list()
    schedules=[]
    array=[]
    next_generation=list()
    cal_fit=dict ()
    for i in range(0,generation_limit):
        population=initialize_population(size,length,Section,Teacher,Room,Course,Day,Duration)
        for i in range(0,int(len(population)/2)):
            a=roulette_wheel(population)
            b=roulette_wheel(population)
            child_a,child_b=single_point_crossover(a,b)
            child_a=mutation(child_a,Section,Teacher,Room,Course,Day,Duration)
            child_b=mutation(child_b,Section,Teacher,Room,Course,Day,Duration)
            next_generation.append(child_a)
            next_generation.append(child_b)
        population=next_generation
    
    for i in range(0,len(population)):
        f=evaluate_fitness(population[i])
        cal_fit[i]=f
    
    sorted_pop = dict (sorted(cal_fit.items(), key=operator.itemgetter(1)))
    k = Counter(sorted_pop) 
    optimal_solution = k.most_common(1)
    for i in optimal_solution: 
        return (population[i[0]])


In [ ]:
def display(solution,length):
    table=PrettyTable(["Professor","Course","Section","Day","Duration(Start time)","Room No"])
    for i in range(length):
        temp=solution[i]
        table.add_row([temp[0],temp[1],temp[2],temp[3],temp[4],temp[5]])
    print(table)


In [ ]:
size=8   #size of population or schedules
length=7 # length of chromosome or one schedule
generation_limit=10 
solution=genetic_algorithm(size,length,generation_limit) #final optimal solution with the greatest fitness value
display(solution,length)


+------------------------+------------------------+---------+----------+----------------------+---------+
|       Professor        |         Course         | Section |   Day    | Duration(Start time) | Room No |
+------------------------+------------------------+---------+----------+----------------------+---------+
|    T4- Dr Omer Beg     |   160 Bioinformatics   |    E    | Thursday |       04:00 PM       |    R1   |
|  T1- Sir Bilal Khalid  | 331 Intro to Computing |    A    | Tuesday  |       04:00 PM       |    R4   |
|  T2- Sir Kashif Munir  | 331 Intro to Computing |    C    |  Monday  |       09:30 AM       |    R4   |
|    T4- Dr Omer Beg     |      440 English       |    A    | Tuesday  |       03:00 PM       |    R2   |
| T3- Maam Amna Basharat |  335 Applied Physics   |    D    | Thursday |       12:00 PM       |    R3   |
|  T1- Sir Bilal Khalid  |      440 English       |    D    |  Friday  |       08:30 AM       |    R4   |
|  T2- Sir Kashif Munir  |       330 Maths    